In [2]:
import numpy as np
import pandas as pd
import pybaseball
import seaborn as sns
import matplotlib.pyplot as plot
from deepdiff import DeepDiff
from pybaseball import bwar_pitch
from pybaseball import bwar_bat
from pybaseball import cache
from pybaseball.lahman import *
from pybaseball import chadwick_register
from pybaseball import pitching_stats
from pybaseball import batting_stats
from time import sleep
cache.enable()
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

In [3]:
people = pd.DataFrame(people())

sleep(1)  
# sleep calls to prevent the pybaseball scraper 
# from throwing errors when importing tons of stuff

people = people.convert_dtypes() # cleanup; thank you NumPy

people = people[people.weight.notna()]

print(people.shape)
print("---")
people.sample(3)

(19729, 24)
---


,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
18150,syndeno01,1992,8,29,USA,TX,Mansfield,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Noah,Syndergaard,Noah Seth,242,78,L,R,2015-05-12,2022-06-27,syndn001,syndeno01
6242,furilca01,1922,3,8,USA,PA,Stony Creek Mills,1989,1,21,USA,PA,Stony Creek Mills,Carl,Furillo,Carl Anthony,190,72,R,R,1946-04-16,1960-05-07,furic101,furilca01
5576,farmebu01,1991,2,20,USA,GA,Conyers,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Buck,Farmer,George Runie,232,76,L,R,2014-08-13,2022-05-04,farmb001,farmebu01


In [4]:
rosetta = pd.DataFrame(chadwick_register()) 

sleep(1)

rosetta = rosetta.convert_dtypes()

rosetta = rosetta.dropna(how='any')

print(rosetta.shape)
print('---')
rosetta.sample(3)

(20506, 8)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
6539,Evers,Tom,113951,evert101,eversto01,1003878,1882,1884
1622,Betts,Harry,110960,betth102,bettsha01,1000932,1903,1913
18762,Rooney,Frank,121438,roonf101,roonefr01,1011186,1914,1914


In [5]:
everyone = pd.merge(rosetta, people, left_on='key_bbref', right_on='playerID', how='left')

print(everyone.shape)
print('---')
everyone.sample(3)

(20506, 32)
---


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
12328,Medina,Luis,118901,medil001,medinlu01,1008715,1988,1991,medinlu01,1963,3,26,USA,CA,Santa Monica,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Luis,Medina,Luis Main,200,76,R,L,1988-09-02,1991-06-13,medil001,medinlu01
10633,Leifield,Lefty,117645,leifl101,leifile01,1007494,1905,1920,leifile01,1883,9,5,USA,IL,Trenton,1970,10,10,USA,VA,Fairfax,Lefty,Leifield,Albert Peter,165,73,L,L,1905-09-03,1920-09-24,leifl101,leifile01
16325,Sanguillen,Manny,121686,sangm101,sanguma01,1011434,1967,1980,sanguma01,1944,3,21,Panama,Colon,Colon,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Manny,Sanguillen,Manuel de Jesus,193,72,R,R,1967-07-23,1980-10-05,sangm101,sanguma01


In [6]:
# fielding stats by year 
fielding = pd.DataFrame(fielding()) # this is fine

fielding = fielding.convert_dtypes()

In [7]:
fielding = fielding.rename(columns={"playerID": "key_bbref"})

In [8]:
everyone = everyone[
    [
        "name_last",
        "name_first",
        "key_mlbam",
        "key_retro",
        "key_bbref",
        "key_fangraphs",
        "birthYear",
        "deathYear",
        "weight",
        "height",
        "bats",
        "throws",
    ]
]

In [9]:
df = everyone
df = fielding.merge(df, on='key_bbref', how='outer', validate='many_to_one')

In [11]:
# Constants
KG_TO_LB = 0.453592
M_TO_IN = 0.0254

In [12]:
# BMI Calculations
df['KG'] = df['weight'] * KG_TO_LB
df['meters'] = df['height'] * M_TO_IN
df['BMI'] = df['KG'] / df['meters'] ** 2
df['ratio'] = df['meters'] * df['BMI']

In [17]:
df = df.drop_duplicates(subset=['key_bbref'])

In [18]:
tryouts = df[ # Let's separate the meat from the chaff
    (df.BMI >= df.BMI.quantile(0.99))
    & (df.height <= df.height.quantile(0.23))]

In [19]:
tryouts.POS.value_counts()

P     17
C     14
1B    10
OF     6
2B     4
SS     1
Name: POS, dtype: Int64

In [21]:
# Probably easiest if we make ourselves a dataframe for every position, since we'll need to fill up a 26-man roster, which is parsed into several discrete roles that each require a particular number of players. 

# These role counts aren't codified, but teams have nearly always carried the same distribution of player-roles.

# The standard breakdown is:
# 13 Pitchers (5x SP, 7x RP)
# 5 OF
# 2 C
# 4 IF
# 2 Utility

# For Utility players, we'll likely take two extra middle infielders, because middle infielders can usually play any field position without looking like they have no idea what they're doing. E.g., you can put a shortstop at first base, and he'll be a pretty mediocre-to-bad first baseman — but if you put a first baseman at shortstop, he's likely to spend the entire game crying.

tryouts_P = tryouts[tryouts['POS'] == "P"]
tryouts_C = tryouts[tryouts['POS'] == "C"]
tryouts_1B = tryouts[tryouts['POS'] == "1B"]
tryouts_2B = tryouts[tryouts['POS'] == "2B"]
tryouts_3B = tryouts[tryouts['POS'] == "3B"]
tryouts_SS = tryouts[tryouts['POS'] == "SS"]
tryouts_OF = tryouts[tryouts['POS'] == "OF"]

In [22]:
test1 = tryouts_P.sort_values(["BMI"], ascending=False).nlargest(13, "BMI")
test1.head(7)

,key_bbref,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,name_last,name_first,key_mlbam,key_retro,key_fangraphs,birthYear,deathYear,weight,height,bats,throws,KG,meters,BMI,ratio
110435,colonba01,1997,1,CLE,AL,P,19,17,282,6,17,5,3,<NA>,<NA>,<NA>,<NA>,<NA>,Colon,Bartolo,112526,colob001,375,1973,<NA>,285,71,R,R,129.27372,1.8034,39.748992,71.683331
143687,moronre01,2017,1,SFN,NL,P,7,0,20,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Moronta,Reyes,606625,moror001,14966,1993,<NA>,265,70,R,R,120.20188,1.778,38.02312,67.605107
129337,mijarjo01,2008,1,MIN,AL,P,10,0,31,0,2,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Mijares,Jose,467726,mijaj001,4140,1984,<NA>,265,71,L,L,120.20188,1.8034,36.959589,66.652922
3211,healeto01,1878,1,PRO,NL,P,3,<NA>,72,0,5,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Healey,Tom,115660,healt101,1005544,1853,1891,155,55,<NA>,R,70.30676,1.397,36.025023,50.326958
136199,machije01,2012,1,SFN,NL,P,8,0,20,0,2,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Machi,Jean,466948,machj002,3220,1982,<NA>,257,71,R,R,116.573144,1.8034,35.843828,64.640759
140236,castran01,2015,1,OAK,AL,P,5,0,12,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Castro,Angel,502162,casta004,3243,1982,<NA>,240,71,R,R,108.86208,1.8034,33.472835,60.364911
146322,colined01,2020,1,MIN,AL,P,1,0,1,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,Colina,Edwar,666674,colie001,20589,1997,<NA>,240,71,R,R,108.86208,1.8034,33.472835,60.364911


In [25]:
bartolo = pd.read_csv(Year,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Awards,Year-additional
1997,24,CLE,AL,4,7,.364,5.65,19,17,0,1,0,0,94.0,107,66,59,12,45,1,66,3,0,5,427,83,4.90,1.617,10.2,1.1,4.3,6.3,1.47,,-9999
1998,25,CLE,AL,14,9,.609,3.71,31,31,0,6,2,0,204.0,205,91,84,15,79,5,158,3,0,4,883,128,3.75,1.392,9.0,0.7,3.5,7.0,2.00,AS,-9999
1999,26,CLE,AL,18,5,.783,3.95,32,32,0,1,1,0,205.0,185,97,90,24,76,5,161,7,0,4,858,126,4.30,1.273,8.1,1.1,3.3,7.1,2.12,CYA-4,-9999
2000,27,CLE,AL,15,8,.652,3.88,30,30,0,2,1,0,188.0,163,86,81,21,98,4,212,4,0,4,807,127,3.96,1.388,7.8,1.0,4.7,10.1,2.16,,-9999
2001,28,CLE,AL,14,12,.538,4.09,34,34,0,1,0,0,222.1,220,106,101,26,90,2,201,2,1,4,947,110,4.00,1.394,8.9,1.1,3.6,8.1,2.23,,-9999
2002,29,TOT,MLB,20,8,.714,2.93,33,33,0,8,3,0,233.1,219,85,76,20,70,5,149,2,0,4,966,147,3.72,1.239,8.4,0.8,2.7,5.7,2.13,CYA-6,-9999
2002,29,CLE,AL,10,4,.714,2.55,16,16,0,4,2,0,116.1,104,37,33,11,31,1,75,2,0,3,467,172,3.75,1.160,8.0,0.9,2.4,5.8,2.42,,-9999
2002,29,MON,NL,10,4,.714,3.31,17,17,0,4,1,0,117.0,115,48,43,9,39,4,74,0,0,1,499,129,3.70,1.316,8.8,0.7,3.0,5.7,1.90,,-9999
2003,30,CHW,AL,15,13,.536,3.87,34,34,0,9,0,0,242.0,223,107,104,30,67,3,173,5,3,8,984,120,4.11,1.198,8.3,1.1,2.5,6.4,2.58,,-9999
2004,31,ANA,AL,18,12,.600,5.01,34,34,0,0,0,0,208.1,215,122,116,38,71,1,158,3,0,1,897,89,4.97,1.373,9.3,1.6,3.1,6.8,2.23,,-9999
2005,32,LAA,AL,21,8,.724,3.48,33,33,0,2,0,0,222.2,215,93,86,26,43,0,157,3,1,2,906,122,3.75,1.159,8.7,1.1,1.7,6.3,3.65,ASCYA-1MVP-23,-9999
2006,33,LAA,AL,1,5,.167,5.11,10,10,0,1,1,0,56.1,71,39,32,11,11,0,31,3,0,1,251,89,5.33,1.456,11.3,1.8,1.8,5.0,2.82,,-9999
2007,34,LAA,AL,6,8,.429,6.34,19,18,0,0,0,0,99.1,132,74,70,15,29,1,76,5,0,1,453,71,4.70,1.621,12.0,1.4,2.6,6.9,2.62,,-9999
2008,35,BOS,AL,4,2,.667,3.92,7,7,0,0,0,0,39.0,44,23,17,5,10,0,27,2,0,0,173,119,4.34,1.385,10.2,1.2,2.3,6.2,2.70,,-9999
2009,36,CHW,AL,3,6,.333,4.19,12,12,0,0,0,0,62.1,69,42,29,13,21,3,38,2,0,1,276,113,5.70,1.444,10.0,1.9,3.0,5.5,1.81,,-9999
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-9999
2011,38,NYY,AL,8,10,.444,4.00,29,26,0,1,1,0,164.1,172,85,73,21,40,3,135,3,0,0,694,107,3.83,1.290,9.4,1.2,2.2,7.4,3.38,,-9999
2012,39,OAK,AL,10,9,.526,3.43,24,24,0,0,0,0,152.1,161,62,58,17,23,3,91,1,0,0,636,114,3.82,1.208,9.5,1.0,1.4,5.4,3.96,,-9999
2013,40,OAK,AL,18,6,.750,2.65,30,30,0,3,3,0,190.1,193,60,56,14,29,0,117,0,0,1,769,147,3.23,1.166,9.1,0.7,1.4,5.5,4.03,ASCYA-6,-9999
2014,41,NYM,NL,15,13,.536,4.09,31,31,0,0,0,0,202.1,218,97,92,22,30,3,151,5,0,2,846,84,3.57,1.226,9.7,1.0,1.3,6.7,5.03,,-9999
2015,42,NYM,NL,14,13,.519,4.16,33,31,1,1,1,0,194.2,217,94,90,25,24,5,136,4,0,0,815,91,3.84,1.238,10.0,1.2,1.1,6.3,5.67,,-9999
2016,43,NYM,NL,15,8,.652,3.43,34,33,0,0,0,0,191.2,200,81,73,24,32,2,128,3,0,0,791,117,3.99,1.210,9.4,1.1,1.5,6.0,4.00,AS,-9999
2017,44,TOT,MLB,7,14,.333,6.48,28,28,0,1,0,0,143.0,192,112,103,28,35,0,89,1,1,1,648,68,5.21,1.587,12.1,1.8,2.2,5.6,2.54,,-9999
2017,44,ATL,NL,2,8,.200,8.14,13,13,0,0,0,0,63.0,92,66,57,11,20,0,42,1,0,1,299,54,5.09,1.778,13.1,1.6,2.9,6.0,2.10,,-9999
2017,44,MIN,AL,5,6,.455,5.18,15,15,0,1,0,0,80.0,100,46,46,17,15,0,47,0,1,0,349,86,5.31,1.438,11.3,1.9,1.7,5.3,3.13,,-9999
2018,45,TEX,AL,7,12,.368,5.78,28,24,1,1,0,0,146.1,172,97,94,32,25,1,81,3,0,2,628,82,5.47,1.346,10.6,2.0,1.5,5.0,3.24,,-9999
21 Yrs,,,,247,188,.568,4.12,565,552,2,38,13,0,3461.2,3593,1719,1584,439,948,47,2535,64,6,45,14655,106,4.15,1.312,9.3,1.1,2.5,6.6,2.67,,-9999
162 Game Avg.,,,,15,11,.568,4.12,34,34,0,2,1,0,211,219,105,96,27,58,3,154,4,0,3,892,106,4.15,1.312,9.3,1.1,2.5,6.6,2.67,,pitch_glossary.shtml#162_avg
CLE (6 yrs),,,,75,45,.625,3.92,162,160,0,15,6,0,1029.2,984,483,448,109,419,18,873,21,1,24,4389,121,4.06,1.363,8.6,1.0,3.7,7.6,2.08,,-9999
LAA (4 yrs),,,,46,33,.582,4.66,96,95,0,3,1,0,586.2,633,328,304,90,154,2,422,14,1,5,2507,94,4.49,1.341,9.7,1.4,2.4,6.5,2.74,,-9999
NYM (3 yrs),,,,44,34,.564,3.90,98,95,1,1,1,0,588.2,635,272,255,71,86,10,415,12,0,2,2452,96,3.79,1.225,9.7,1.1,1.3,6.3,4.83,,-9999
OAK (2 yrs),,,,28,15,.651,2.99,54,54,0,3,3,0,342.2,354,122,114,31,52,3,208,1,0,1,1405,130,3.49,1.185,9.3,0.8,1.4,5.5,4.00,,-9999
CHW (2 yrs),,,,18,19,.486,3.93,46,46,0,9,0,0,304.1,292,149,133,43,88,6,211,7,3,9,1260,118,4.43,1.249,8.6,1.3,2.6,6.2,2.40,,-9999
MIN (1 yr),,,,5,6,.455,5.18,15,15,0,1,0,0,80.0,100,46,46,17,15,0,47,0,1,0,349,86,5.31,1.438,11.3,1.9,1.7,5.3,3.13,,-9999
ATL (1 yr),,,,2,8,.200,8.14,13,13,0,0,0,0,63.0,92,66,57,11,20,0,42,1,0,1,299,54,5.09,1.778,13.1,1.6,2.9,6.0,2.10,,-9999
BOS (1 yr),,,,4,2,.667,3.92,7,7,0,0,0,0,39.0,44,23,17,5,10,0,27,2,0,0,173,119,4.34,1.385,10.2,1.2,2.3,6.2,2.70,,-9999
TEX (1 yr),,,,7,12,.368,5.78,28,24,1,1,0,0,146.1,172,97,94,32,25,1,81,3,0,2,628,82,5.47,1.346,10.6,2.0,1.5,5.0,3.24,,-9999
MON (1 yr),,,,10,4,.714,3.31,17,17,0,4,1,0,117.0,115,48,43,9,39,4,74,0,0,1,499,129,3.70,1.316,8.8,0.7,3.0,5.7,1.90,,-9999
NYY (1 yr),,,,8,10,.444,4.00,29,26,0,1,1,0,164.1,172,85,73,21,40,3,135,3,0,0,694,107,3.83,1.290,9.4,1.2,2.2,7.4,3.38,,-9999
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-9999
AL (18 yrs),,,,191,142,.574,4.11,437,427,1,33,11,0,2693.0,2751,1333,1229,348,803,33,2004,51,6,41,11405,110,4.23,1.320,9.2,1.2,2.7,6.7,2.50,,-9999
NL (5 yrs),,,,56,46,.549,4.16,128,125,1,5,2,0,768.2,842,386,355,91,145,14,531,13,0,4,3250,93,3.89,1.284,9.9,1.1,1.7,6.2,3.66,,-9999

SyntaxError: invalid decimal literal (1745309096.py, line 12)

In [23]:
Year,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Awards,Year-additional
1997,24,CLE,AL,4,7,.364,5.65,19,17,0,1,0,0,94.0,107,66,59,12,45,1,66,3,0,5,427,83,4.90,1.617,10.2,1.1,4.3,6.3,1.47,,-9999
1998,25,CLE,AL,14,9,.609,3.71,31,31,0,6,2,0,204.0,205,91,84,15,79,5,158,3,0,4,883,128,3.75,1.392,9.0,0.7,3.5,7.0,2.00,AS,-9999
1999,26,CLE,AL,18,5,.783,3.95,32,32,0,1,1,0,205.0,185,97,90,24,76,5,161,7,0,4,858,126,4.30,1.273,8.1,1.1,3.3,7.1,2.12,CYA-4,-9999
2000,27,CLE,AL,15,8,.652,3.88,30,30,0,2,1,0,188.0,163,86,81,21,98,4,212,4,0,4,807,127,3.96,1.388,7.8,1.0,4.7,10.1,2.16,,-9999
2001,28,CLE,AL,14,12,.538,4.09,34,34,0,1,0,0,222.1,220,106,101,26,90,2,201,2,1,4,947,110,4.00,1.394,8.9,1.1,3.6,8.1,2.23,,-9999
2002,29,TOT,MLB,20,8,.714,2.93,33,33,0,8,3,0,233.1,219,85,76,20,70,5,149,2,0,4,966,147,3.72,1.239,8.4,0.8,2.7,5.7,2.13,CYA-6,-9999
2002,29,CLE,AL,10,4,.714,2.55,16,16,0,4,2,0,116.1,104,37,33,11,31,1,75,2,0,3,467,172,3.75,1.160,8.0,0.9,2.4,5.8,2.42,,-9999
2002,29,MON,NL,10,4,.714,3.31,17,17,0,4,1,0,117.0,115,48,43,9,39,4,74,0,0,1,499,129,3.70,1.316,8.8,0.7,3.0,5.7,1.90,,-9999
2003,30,CHW,AL,15,13,.536,3.87,34,34,0,9,0,0,242.0,223,107,104,30,67,3,173,5,3,8,984,120,4.11,1.198,8.3,1.1,2.5,6.4,2.58,,-9999
2004,31,ANA,AL,18,12,.600,5.01,34,34,0,0,0,0,208.1,215,122,116,38,71,1,158,3,0,1,897,89,4.97,1.373,9.3,1.6,3.1,6.8,2.23,,-9999
2005,32,LAA,AL,21,8,.724,3.48,33,33,0,2,0,0,222.2,215,93,86,26,43,0,157,3,1,2,906,122,3.75,1.159,8.7,1.1,1.7,6.3,3.65,ASCYA-1MVP-23,-9999
2006,33,LAA,AL,1,5,.167,5.11,10,10,0,1,1,0,56.1,71,39,32,11,11,0,31,3,0,1,251,89,5.33,1.456,11.3,1.8,1.8,5.0,2.82,,-9999
2007,34,LAA,AL,6,8,.429,6.34,19,18,0,0,0,0,99.1,132,74,70,15,29,1,76,5,0,1,453,71,4.70,1.621,12.0,1.4,2.6,6.9,2.62,,-9999
2008,35,BOS,AL,4,2,.667,3.92,7,7,0,0,0,0,39.0,44,23,17,5,10,0,27,2,0,0,173,119,4.34,1.385,10.2,1.2,2.3,6.2,2.70,,-9999
2009,36,CHW,AL,3,6,.333,4.19,12,12,0,0,0,0,62.1,69,42,29,13,21,3,38,2,0,1,276,113,5.70,1.444,10.0,1.9,3.0,5.5,1.81,,-9999
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-9999
2011,38,NYY,AL,8,10,.444,4.00,29,26,0,1,1,0,164.1,172,85,73,21,40,3,135,3,0,0,694,107,3.83,1.290,9.4,1.2,2.2,7.4,3.38,,-9999
2012,39,OAK,AL,10,9,.526,3.43,24,24,0,0,0,0,152.1,161,62,58,17,23,3,91,1,0,0,636,114,3.82,1.208,9.5,1.0,1.4,5.4,3.96,,-9999
2013,40,OAK,AL,18,6,.750,2.65,30,30,0,3,3,0,190.1,193,60,56,14,29,0,117,0,0,1,769,147,3.23,1.166,9.1,0.7,1.4,5.5,4.03,ASCYA-6,-9999
2014,41,NYM,NL,15,13,.536,4.09,31,31,0,0,0,0,202.1,218,97,92,22,30,3,151,5,0,2,846,84,3.57,1.226,9.7,1.0,1.3,6.7,5.03,,-9999
2015,42,NYM,NL,14,13,.519,4.16,33,31,1,1,1,0,194.2,217,94,90,25,24,5,136,4,0,0,815,91,3.84,1.238,10.0,1.2,1.1,6.3,5.67,,-9999
2016,43,NYM,NL,15,8,.652,3.43,34,33,0,0,0,0,191.2,200,81,73,24,32,2,128,3,0,0,791,117,3.99,1.210,9.4,1.1,1.5,6.0,4.00,AS,-9999
2017,44,TOT,MLB,7,14,.333,6.48,28,28,0,1,0,0,143.0,192,112,103,28,35,0,89,1,1,1,648,68,5.21,1.587,12.1,1.8,2.2,5.6,2.54,,-9999
2017,44,ATL,NL,2,8,.200,8.14,13,13,0,0,0,0,63.0,92,66,57,11,20,0,42,1,0,1,299,54,5.09,1.778,13.1,1.6,2.9,6.0,2.10,,-9999
2017,44,MIN,AL,5,6,.455,5.18,15,15,0,1,0,0,80.0,100,46,46,17,15,0,47,0,1,0,349,86,5.31,1.438,11.3,1.9,1.7,5.3,3.13,,-9999
2018,45,TEX,AL,7,12,.368,5.78,28,24,1,1,0,0,146.1,172,97,94,32,25,1,81,3,0,2,628,82,5.47,1.346,10.6,2.0,1.5,5.0,3.24,,-9999
21 Yrs,,,,247,188,.568,4.12,565,552,2,38,13,0,3461.2,3593,1719,1584,439,948,47,2535,64,6,45,14655,106,4.15,1.312,9.3,1.1,2.5,6.6,2.67,,-9999
162 Game Avg.,,,,15,11,.568,4.12,34,34,0,2,1,0,211,219,105,96,27,58,3,154,4,0,3,892,106,4.15,1.312,9.3,1.1,2.5,6.6,2.67,,pitch_glossary.shtml#162_avg
CLE (6 yrs),,,,75,45,.625,3.92,162,160,0,15,6,0,1029.2,984,483,448,109,419,18,873,21,1,24,4389,121,4.06,1.363,8.6,1.0,3.7,7.6,2.08,,-9999
LAA (4 yrs),,,,46,33,.582,4.66,96,95,0,3,1,0,586.2,633,328,304,90,154,2,422,14,1,5,2507,94,4.49,1.341,9.7,1.4,2.4,6.5,2.74,,-9999
NYM (3 yrs),,,,44,34,.564,3.90,98,95,1,1,1,0,588.2,635,272,255,71,86,10,415,12,0,2,2452,96,3.79,1.225,9.7,1.1,1.3,6.3,4.83,,-9999
OAK (2 yrs),,,,28,15,.651,2.99,54,54,0,3,3,0,342.2,354,122,114,31,52,3,208,1,0,1,1405,130,3.49,1.185,9.3,0.8,1.4,5.5,4.00,,-9999
CHW (2 yrs),,,,18,19,.486,3.93,46,46,0,9,0,0,304.1,292,149,133,43,88,6,211,7,3,9,1260,118,4.43,1.249,8.6,1.3,2.6,6.2,2.40,,-9999
MIN (1 yr),,,,5,6,.455,5.18,15,15,0,1,0,0,80.0,100,46,46,17,15,0,47,0,1,0,349,86,5.31,1.438,11.3,1.9,1.7,5.3,3.13,,-9999
ATL (1 yr),,,,2,8,.200,8.14,13,13,0,0,0,0,63.0,92,66,57,11,20,0,42,1,0,1,299,54,5.09,1.778,13.1,1.6,2.9,6.0,2.10,,-9999
BOS (1 yr),,,,4,2,.667,3.92,7,7,0,0,0,0,39.0,44,23,17,5,10,0,27,2,0,0,173,119,4.34,1.385,10.2,1.2,2.3,6.2,2.70,,-9999
TEX (1 yr),,,,7,12,.368,5.78,28,24,1,1,0,0,146.1,172,97,94,32,25,1,81,3,0,2,628,82,5.47,1.346,10.6,2.0,1.5,5.0,3.24,,-9999
MON (1 yr),,,,10,4,.714,3.31,17,17,0,4,1,0,117.0,115,48,43,9,39,4,74,0,0,1,499,129,3.70,1.316,8.8,0.7,3.0,5.7,1.90,,-9999
NYY (1 yr),,,,8,10,.444,4.00,29,26,0,1,1,0,164.1,172,85,73,21,40,3,135,3,0,0,694,107,3.83,1.290,9.4,1.2,2.2,7.4,3.38,,-9999
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-9999
AL (18 yrs),,,,191,142,.574,4.11,437,427,1,33,11,0,2693.0,2751,1333,1229,348,803,33,2004,51,6,41,11405,110,4.23,1.320,9.2,1.2,2.7,6.7,2.50,,-9999
NL (5 yrs),,,,56,46,.549,4.16,128,125,1,5,2,0,768.2,842,386,355,91,145,14,531,13,0,4,3250,93,3.89,1.284,9.9,1.1,1.7,6.2,3.66,,-9999


SyntaxError: invalid decimal literal (2366721775.py, line 12)